In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D
from keras.layers import BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers.merge import Concatenate
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from keras.preprocessing.image import ImageDataGenerator
import keras
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime as dt
import cv2
data_dir = './' #Change to your directory here

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
def load_data(data_dir):
    train = pd.read_json(data_dir + 'train.json')
    print("Train data loaded")
    test = pd.read_json(data_dir + 'test.json')
    print("Test data loaded")
    #Fill 'na' angles with mode
    train.inc_angle = train.inc_angle.replace('na', 0)
    train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
    test.inc_angle = test.inc_angle.replace('na', 0)
    test.inc_angle = test.inc_angle.astype(float).fillna(0.0)
    return train, test

train, test = load_data(data_dir)
print("done")

X_angle_train = np.array(train.inc_angle)
X_angle_test = np.array(test.inc_angle)


def color_composite(data):
    w,h = 75,75
    rgb_arrays = []
    for i, row in data.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2
        band_4 = band_2 * row['inc_angle']


        r = (band_1 + abs(band_1.min())) / np.max((band_1 + abs(band_1.min())))
        g = (band_2 + abs(band_2.min())) / np.max((band_2 + abs(band_2.min())))
        b = (band_3 + abs(band_3.min())) / np.max((band_3 + abs(band_3.min())))
        a = (band_3 + abs(band_3.min())) / np.max((band_3 + abs(band_3.min())))

        
        rgba = np.dstack((r, g, b))

        #rgba = cv2.resize(rgba, (w,h)).astype(np.float32)
        
        rgb_arrays.append(rgba)
    return np.array(rgb_arrays)

rgb_train = color_composite(train)
rgb_test = color_composite(test)
print("RGBs Done")

y_train = np.array(train['is_iceberg'])

from sklearn.model_selection import train_test_split

X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(rgb_train,
                                    X_angle_train, y_train, random_state=420, test_size=0.25)

print("X_train:",X_train.shape)
print("X_valid:",X_valid.shape)

Train data loaded
Test data loaded
done
RGBs Done
X_train: (1203, 75, 75, 3)
X_valid: (401, 75, 75, 3)


In [3]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value.
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(name = bn_name_base + '2a')(X)
    X = Activation('elu')(X)
    
    
    # Second component of main path 
    X = Conv2D(F2, kernel_size=(f,f), strides=(1,1), padding='same', name= conv_name_base+'2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(name = bn_name_base + '2b')(X)
    X = Activation('elu')(X)

    # Third component of main path 
    X = Conv2D(F3, kernel_size=(1,1), strides=(1,1), padding='valid', name= conv_name_base+'2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('elu')(X)
    
    return X

In [4]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(name = bn_name_base + '2a')(X)
    X = Activation('elu')(X)


    # Second component of main path
    X = Conv2D(F2, kernel_size=(f,f), strides=(1,1), padding='same', name=conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base+'2b')(X)
    X = Activation('elu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(name=bn_name_base+'2c')(X)

    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('elu')(X)

    
    return X

In [8]:
def ResNet50(input_shape = (75, 75, 3), classes = 1):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")

    
    # Zero-Padding
    X = ZeroPadding2D((0, 0))(input_1)
    
    # Stage 1
    X = Conv2D(16, (3, 3), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(name = 'bn_conv1')(X)
    X = Activation('elu')(X)
   # X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [16, 16, 64], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [16, 16, 64], stage=2, block='b')
    X = identity_block(X, 3, [16, 16, 64], stage=2, block='c')
    X = identity_block(X, 3, [16, 16, 64], stage=2, block='d')



    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [32, 32, 128], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [32, 32, 128], stage=3, block='b')
    X = identity_block(X, 3, [32,32, 128], stage=3, block='c')
    X = identity_block(X, 3, [32,32, 128], stage=3, block='d')
    X = identity_block(X, 3, [32, 32, 128], stage=3, block='e')
    X = identity_block(X, 3, [32,32, 128], stage=3, block='f')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [64, 64, 256], stage=4, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=4, block='c')
    X = identity_block(X, 3, [64, 64, 256], stage=4, block='d')

    
    # AVGPOOL
    X = AveragePooling2D(name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Concatenate()([X, BatchNormalization()(input_2)])
    X = Dense(classes, activation='sigmoid', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = [input_1, input_2], outputs = X, name='ResNet50')

    return model

In [9]:
from keras import backend as K
K.clear_session()

model = ResNet50()


print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_1 (InputLayer)                (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 75, 75, 3)    0           X_1[0][0]                        
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 37, 37, 16)   448         zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 37, 37, 16)   64          conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [ ]:
batch_size = 32
#Lets define the image transormations that we want
gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.1,
                         rotation_range=90)

# Here is the function that merges our two generators
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=12)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=12)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
gen_flow = gen_flow_for_two_inputs(X_train, X_angle_train, y_train)
gen_flow_valid = gen_flow_for_two_inputs(X_valid, X_angle_valid, y_valid)


from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
#call backs
weights_file = './model.h5'

earlystop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, min_delta=1e-4, mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=40, verbose=1, epsilon=1e-4, mode='min')
model_chk = ModelCheckpoint(monitor='val_loss', filepath=weights_file, save_best_only=True, 
                            save_weights_only=True, mode='min')
        
callbacks = [earlystop, reduce_lr_loss, model_chk, TensorBoard(log_dir='./logs1')]

model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.002), 
              loss='mae', metrics=['accuracy'])

#fit the model
model.fit_generator(gen_flow, validation_data=gen_flow_valid, 
                    validation_steps=np.ceil((7 * float(len(y_train)) / float(batch_size))/10), 
                    steps_per_epoch=np.ceil(7 * float(len(y_train)) / float(batch_size)), 
                    epochs=500, verbose=1, callbacks=callbacks)
#model.fit([X_train, X_angle_train],y_train, validation_data=([X_valid, X_angle_valid], y_valid),  
 #                   epochs=500, verbose=1, callbacks=callbacks)


Epoch 1/500
264/264 [==============================] - 68s 258ms/step - loss: 0.3406 - acc: 0.6593 - val_loss: 0.5276 - val_acc: 0.4724
Epoch 2/500
264/264 [==============================] - 59s 223ms/step - loss: 0.3269 - acc: 0.6728 - val_loss: 0.3206 - val_acc: 0.6799
Epoch 3/500
264/264 [==============================] - 58s 222ms/step - loss: 0.3247 - acc: 0.6752 - val_loss: 0.3070 - val_acc: 0.6942
Epoch 4/500
264/264 [==============================] - 58s 220ms/step - loss: 0.3169 - acc: 0.6835 - val_loss: 0.2785 - val_acc: 0.7230
Epoch 5/500
264/264 [==============================] - 59s 224ms/step - loss: 0.2936 - acc: 0.7064 - val_loss: 0.4996 - val_acc: 0.5000
Epoch 6/500
264/264 [==============================] - 59s 225ms/step - loss: 0.2793 - acc: 0.7205 - val_loss: 0.2470 - val_acc: 0.7530
Epoch 7/500
264/264 [==============================] - 58s 220ms/step - loss: 0.2441 - acc: 0.7557 - val_loss: 0.2376 - val_acc: 0.7656
Epoch 8/500
264/264 [===========================

Epoch 64/500
264/264 [==============================] - 70s 265ms/step - loss: 0.1733 - acc: 0.8263 - val_loss: 0.1724 - val_acc: 0.8273
Epoch 65/500
264/264 [==============================] - 70s 264ms/step - loss: 0.1745 - acc: 0.8255 - val_loss: 0.1539 - val_acc: 0.8462
Epoch 66/500
264/264 [==============================] - 70s 265ms/step - loss: 0.1781 - acc: 0.8225 - val_loss: 0.1491 - val_acc: 0.8525
Epoch 67/500
264/264 [==============================] - 70s 264ms/step - loss: 0.1791 - acc: 0.8202 - val_loss: 0.1628 - val_acc: 0.8381
Epoch 68/500
264/264 [==============================] - 70s 264ms/step - loss: 0.1804 - acc: 0.8200 - val_loss: 0.1433 - val_acc: 0.8537
Epoch 69/500
264/264 [==============================] - 70s 265ms/step - loss: 0.1752 - acc: 0.8253 - val_loss: 0.1427 - val_acc: 0.8549
Epoch 70/500
264/264 [==============================] - 70s 265ms/step - loss: 0.1827 - acc: 0.8174 - val_loss: 0.1636 - val_acc: 0.8369
Epoch 71/500
264/264 [===================

264/264 [==============================] - 71s 269ms/step - loss: 0.1641 - acc: 0.8364 - val_loss: 0.1442 - val_acc: 0.8573
Epoch 132/500
264/264 [==============================] - 69s 261ms/step - loss: 0.1650 - acc: 0.8346 - val_loss: 0.1465 - val_acc: 0.8537
Epoch 133/500
264/264 [==============================] - 70s 267ms/step - loss: 0.1682 - acc: 0.8328 - val_loss: 0.1489 - val_acc: 0.8525
Epoch 134/500
264/264 [==============================] - 71s 268ms/step - loss: 0.1696 - acc: 0.8304 - val_loss: 0.1493 - val_acc: 0.8525
Epoch 135/500
264/264 [==============================] - 71s 269ms/step - loss: 0.1644 - acc: 0.8366 - val_loss: 0.1401 - val_acc: 0.8585
Epoch 136/500
264/264 [==============================] - 71s 270ms/step - loss: 0.1634 - acc: 0.8368 - val_loss: 0.1522 - val_acc: 0.8477
Epoch 137/500
264/264 [==============================] - 71s 268ms/step - loss: 0.1629 - acc: 0.8373 - val_loss: 0.1437 - val_acc: 0.8561
Epoch 138/500
263/264 [=========================

In [ ]:
# Predict on test data
from keras.models import load_model
model.load_weights('model.h5')

print(model.evaluate([X_train,X_angle_train], y_train))
print(model.evaluate([X_valid,X_angle_valid], y_valid))

In [56]:
test_predictions = model.predict([rgb_test, X_angle_test])

tmp = dt.datetime.now().strftime("%Y-%m-%d-%H-%M")
nb_filters = 2
file = 'subm_{}_f{:03d}.csv'.format(tmp, nb_filters)

# Create .csv
pred_df = test[['id']].copy()
pred_df['is_iceberg'] = test_predictions
pred_df.to_csv('./submitions/{}'.format(file), index=False, float_format='%.4f')
pred_df.head()

InvalidArgumentError: You must feed a value for placeholder tensor 'keras_learning_phase' with dtype bool
	 [[Node: keras_learning_phase = Placeholder[dtype=DT_BOOL, shape=[], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: bn_conv1/moments/sufficient_statistics/Gather/_3969 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_627_bn_conv1/moments/sufficient_statistics/Gather", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'keras_learning_phase', defined at:
  File "C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in